In [ ]:
# import packages
import pandas as pd
import os
import sys
import json
import spotipy
import webbrowser
import spotipy.util as util
from json.decoder import JSONDecodeError

In [ ]:
# Set user's credencials to access Spotify data
scope = 'user-read-private user-read-playback-state user-modify-playback-state'

if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print("Usage: %s username" % (sys.argv[0],))
    sys.exit()

token = util.prompt_for_user_token(username = 'your-spotify-username',
                           scope = scope,
                           client_id='your-spotify-client-id',
                           client_secret='your-spotify-client-secret',
                           redirect_uri='your-app-redirect-url')

spotifyObject = spotipy.Spotify(auth=token)

In [ ]:
# Create empty dataframes
df_artist = pd.DataFrame()
df_albums = pd.DataFrame()
df_tracks = pd.DataFrame()

In [ ]:
# Create a empty set 
artists_searched = set()

In [ ]:
# User information
user = spotifyObject.current_user()
displayName = user['display_name']

# PROGRAMME TO EXTRACT SPOTIFY DATA USING SPOTIPY
while True:
    # Main Menu
   
    print("Welcome to Spotipy, " + displayName + "!")
    print()
    print("1 - Search for an artist")
    print("2 - See consulted artists")
    print("0 - exit")
    print()
    choice = input("Choose an option: ")

    if choice == "1":
        print()
        searchQuery = input("Artist name: ")
        print()

        # ARTIST DETAILS
        searchResults = spotifyObject.search(searchQuery,1,0,"artist")

        artist = searchResults['artists']['items'][0]
        artistID = artist['id']
        artists_searched.add(artist['name'])
        
        print(artist['name'])
        print()

        
        ## Extract arist data and fill database
        df_artist = df_artist.append({'artist_id': artist['id'],
                                      'artist_name':artist['name'],
                                      'genres':artist['genres'],
                                      'folowers':artist['followers']['total'],
                                      'popularity':artist['popularity']}, 
                                     ignore_index=True
                                    )
                                      
        # ALBUM DETAILS
        albumResults = spotifyObject.artist_albums(artistID)
        albumResults = albumResults['items']
        z = 0

        # Loop over album
        for album in albumResults:
            
            ## Extract album data and fill database
            df_albums = df_albums.append({'artist_name':album['artists'][0]['name'],
                                          'artist_id':album['artists'][0]['id'],
                                          'album_id': album['id'],
                                         'album_name':album['name'],
                                         'release_date':album['release_date'],
                                         'type':album['type']}, 
                                         ignore_index=True
                                        )
            
            print("ALBUM: " + album['name'])
            albumID = album['id']

            # TRACK DETAILS
            trackResults = spotifyObject.album_tracks(albumID)
            trackResults = trackResults['items']
            
            # Loop ver tracks
            for track in trackResults:
                
                artists_name =[]
                artists_id = []
                z+=1
                for artist in track['artists']:
                    artists_name.append(artist['name'])
             
                for artist_id in track['artists']:
                    artists_id.append(artist_id['id'])
                
                ## Extract track data and fill database
                df_tracks = df_tracks.append({'artist_id': artists_id,  
                                'artists_name': artists_name, 
                                'song_id': track['id'],
                                'song_name':track['name'],
                                }, ignore_index=True
                                )
                print(str(z) + ": " + track['name'])
                
            print()
        print('Number of songs:', z)
        print()
        
        print()
        print("---------------------")
        print()

    if choice == "2":
        if len(artists_searched) == 0:
            print()
            print ('Artists consulted: 0')
            print()
            print("---------------------")
            print()
        
        if len(artists_searched) > 0:
            print()
            print('Artists consulted: ', artists_searched)
            print('Number of artists consulted: ', len(artists_searched))
            print()
            print("---------------------")
            print()
        
    if choice == "0":
        break